# The Confusion Matrix 
## Assignment 6
### Members:
Shruthi Gopinath: 01FB16ECS378  

Vaani Sundaresh: 01FB16ECS429  

Varsha Venkatsubramanian: 01FB16ECS432  


In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
from math import sqrt
import os
from sklearn.cluster import KMeans
from statistics import mode
from collections import Counter
print(os.listdir("../input"))


In [ ]:
df=pd.read_csv('../input/Absenteeism_at_work.csv')
df

In [ ]:
# one-hot encoding for the categorical columns
new_df=df.copy()
lc=[1,2,3,4,12]
for col in range(len(new_df.columns.values)):
    if col in lc:
        newcolnames=[]
        colv=new_df.columns.values[col]
        dummy=pd.get_dummies(new_df[colv])
        colnames=dummy.columns.values
        for i in range(len(colnames)):
            newcolnames.append(colv+": "+str(colnames[i]))
        dummy.columns=newcolnames
        new_df=pd.concat([new_df,dummy], axis=1)

In [ ]:
new_df.pop('ID')
new_df.pop('Reason for absence')
new_df.pop('Month of absence')
new_df.pop('Day of the week')
new_df.pop('Seasons')
p=new_df.pop('Education')

In [ ]:
df_old=df.copy()
df=new_df.copy()

In [ ]:
# preprocessing for stratification
df['Label_Count']=0
x=pd.DataFrame(df.groupby('Absenteeism time in hours').count()['Age'])
for i in range(len(df)):
    label=df.loc[i]['Absenteeism time in hours']
    label=int(label)
    count=x.loc[label]['Age']
    df.loc[i,'Label_Count']=count

In [ ]:
df_temp=df[df['Label_Count']==1]

In [ ]:
for i in range(len(df)):
    if df.loc[i]['Label_Count']==1:
        df=df.drop(i)

In [ ]:
# using stratification to generate training and testing datasets, so as to ensure that the data gets
# split evenly with respect to 'Absenteeism time in hours' values, ie, the target variable
df_train, df_test = train_test_split(df, test_size=0.2, stratify=df['Absenteeism time in hours'])

In [ ]:
df_train=pd.concat([df_train,df_temp])

In [ ]:
df_train=df_train.reset_index(drop=True)
df_test=df_test.reset_index(drop=True)

In [ ]:
# normalizing the features of the datasets
scaled_train = pd.DataFrame(preprocessing.normalize(df_train.loc[:,'Transportation expense':'Body mass index']))
scaled_test = pd.DataFrame(preprocessing.normalize(df_test.loc[:,'Transportation expense':'Body mass index']))

In [ ]:
scaled_train=pd.concat([scaled_train,df_train.loc[:,'Absenteeism time in hours':'Label_Count']],axis=1)
scaled_test=pd.concat([scaled_test,df_test.loc[:,'Absenteeism time in hours':'Label_Count']],axis=1)

In [ ]:
scaled_train.columns=df_train.columns
scaled_test.columns=df_test.columns

In [ ]:
y_train=pd.DataFrame(scaled_train['Absenteeism time in hours'])
y_test=pd.DataFrame(scaled_test['Absenteeism time in hours'])
y_train.columns=['Hours Absent']
y_test.columns=['Hours Absent']


In [ ]:
temp=scaled_train.pop('Absenteeism time in hours')
temp=scaled_test.pop('Absenteeism time in hours')
temp=scaled_train.pop('Label_Count')
temp=scaled_test.pop('Label_Count')

# K Nearest Neighbours Classification
### We iterate over different values of k and choose the optimal value - the one that results in best accuracy. Below is the code, followed by the graph that shows variation of accuracy with increasing values of k.
### The unique values of 'Absenteeism time in hours' (target variable) will be considered as the possible classes.
Note: A disadvantage of using classification for this dataset is that it will never predict a value that doesn't exist as a class already.

In [ ]:
accs=[]
for i in range(1,100):
    classifier = KNeighborsClassifier(n_neighbors=i)
    classifier.fit(scaled_train, np.array(y_train['Hours Absent']))
    y_pred = classifier.predict(scaled_test) 
    result=y_test
    result['Predicted']=y_pred
    count_correst=0
    for i in range(len(result)):
            if result.loc[i]['Hours Absent']==result.loc[i]['Predicted']:
                count_correst+=1
    accuracy=count_correst*100/len(result)
    accs.append(accuracy)

In [ ]:
plt.title('Variation of accuracy with value of k')
plt.xlabel('value of k')
plt.ylabel('accuracy in %')
plt.plot(accs)

In [ ]:
#even if there are multiple occurences of the maximum accuracy, we need to find 
#the lowest k to ensure efficiency
optimal_k=accs.index(max(accs))+1
knn_acc=max(accs)
print("Maximum accuracy is:",max(accs))
print("Optimal value of k for k Nearest Neighbours classification model is:",optimal_k)

In [ ]:
classifier = KNeighborsClassifier(n_neighbors=optimal_k)
classifier.fit(scaled_train, np.array(y_train['Hours Absent']))
y_pred = classifier.predict(scaled_test) 
result=y_test
result['Predicted']=y_pred
rms_knn = sqrt(mean_squared_error(result['Hours Absent'], result['Predicted']))
print('RMS error for k=%d is: %d hours'%(optimal_k,rms_knn))

# K Means Clustering
### We vary the number of clusters and choose the optimal value - the one that results in best accuracy. Below is the code, followed by the graph that shows variation of accuracy with increasing values of k (number of clusters).
### Possible variation: consider the number of clusters as 19, because that is the number of unique values in the 'Absenteeism time in hours' feature, but this often turns out to be less accurate than some other values of k.

In [ ]:
df=pd.read_csv('../input/Absenteeism_at_work.csv')
df

In [ ]:
# one-hot encoding for categorical columns
new_df=df.copy()
lc=[1,2,3,4,12]
for col in range(len(new_df.columns.values)):
    if col in lc:
        newcolnames=[]
        colv=new_df.columns.values[col]
        dummy=pd.get_dummies(new_df[colv])
        colnames=dummy.columns.values
        for i in range(len(colnames)):
            newcolnames.append(colv+": "+str(colnames[i]))
        dummy.columns=newcolnames
        new_df=pd.concat([new_df,dummy], axis=1)
new_df.pop('ID')
new_df.pop('Reason for absence')
new_df.pop('Month of absence')
new_df.pop('Day of the week')
new_df.pop('Seasons')
p=new_df.pop('Education')

In [ ]:
df_old=df.copy()
df=new_df.copy()

In [ ]:
# preprocessing required for stratification
df['Label_Count']=0
x=pd.DataFrame(df.groupby('Absenteeism time in hours').count()['Age'])
for i in range(len(df)):
    label=df.loc[i]['Absenteeism time in hours']
    label=int(label)
    count=x.loc[label]['Age']
    df.loc[i,'Label_Count']=count
df_temp=df[df['Label_Count']==1]
for i in range(len(df)):
    if df.loc[i]['Label_Count']==1:
        df=df.drop(i)

In [ ]:
# splitting into testing and training datasets using stratification
df_train, df_test = train_test_split(df, test_size=0.2, stratify=df['Absenteeism time in hours'])
df_train=pd.concat([df_train,df_temp])
df_train=df_train.reset_index(drop=True)
df_test=df_test.reset_index(drop=True)

# normalizing the features
scaled_train = pd.DataFrame(preprocessing.normalize(df_train.loc[:,'Transportation expense':'Body mass index']))
scaled_test = pd.DataFrame(preprocessing.normalize(df_test.loc[:,'Transportation expense':'Body mass index']))
scaled_train=pd.concat([scaled_train,df_train.loc[:,'Absenteeism time in hours':'Label_Count']],axis=1)
scaled_test=pd.concat([scaled_test,df_test.loc[:,'Absenteeism time in hours':'Label_Count']],axis=1)                            
scaled_train.columns=df_train.columns
scaled_test.columns=df_test.columns
p=scaled_train.pop('Label_Count')
p=scaled_test.pop('Label_Count')

In [ ]:
df2=scaled_train.copy()
df3=df2.copy()
p=df3.pop('Absenteeism time in hours')
df4=scaled_test.copy()
abst=df4.pop('Absenteeism time in hours')

In [ ]:
# K-Means clustering to predict 'Absenteeism time in hours' values for the testing data
accs=[]
for kval in range(3,30):
    kmeans=KMeans(n_clusters=kval).fit(df3)
    centroid=kmeans.cluster_centers_
    labels=kmeans.labels_
    df3['Labels']=labels
    #each list within cluster_values represents a cluster and has the Absenteeism time in hours for all 
    #the data points in that cluster
    cluster_values=[]
    for i in range(kval):
        cluster_values.append([])
    for i in range(len(df3)):
        cluster_values[int(df3.loc[i][-1])].append(df2.loc[i][8])
    #finding mode of Absenteeism time in hours for each cluster
    cluster_modes=[]
    for i in cluster_values:
        cluster_modes.append((Counter(i).most_common(1))[0][0])
    for i in range(len(df3)):
        df3.loc[i,'Labels']=cluster_modes[int(df3.loc[i][-1])]
    df4['Labels']=kmeans.predict(df4)
    for i in range(len(df4)):
        df4.loc[i,'Labels']=cluster_modes[int(df4.loc[i][-1])]
    count_right=0
    for (i,j) in zip(df4.values,scaled_test.values):
        if i[-1]==j[8]:
            count_right+=1
    test_acc=count_right*100/len(df4)
    accs.append(test_acc)
    x=df3.pop('Labels')
    x=df4.pop('Labels')

In [ ]:
# plotting the accuracy percentages for different k values
plt.title('Variation of accuracy with value of k')
plt.xlabel('value of k')
plt.ylabel('accuracy in %')
plt.plot(accs)

In [ ]:
kmeans_acc=max(accs)
print("Maximum testing accuracy:",max(accs))
optimal_k=accs.index(max(accs))+1
print("Optimal value of k:",optimal_k)

In [ ]:
kmeans=KMeans(n_clusters=optimal_k).fit(df3)
centroid=kmeans.cluster_centers_
labels=kmeans.labels_
df3['Labels']=labels
#each list within cluster_values represents a cluster and has the Absenteeism time in hours for all 
#the data points in that cluster
cluster_values=[]
for i in range(optimal_k):
    cluster_values.append([])
for i in range(len(df3)):
    cluster_values[int(df3.loc[i][-1])].append(df2.loc[i][8])
#finding mode of Absenteeism time in hours for each cluster
cluster_modes=[]
for i in cluster_values:
    cluster_modes.append((Counter(i).most_common(1))[0][0])
for i in range(len(df3)):
    df3.loc[i,'Labels']=cluster_modes[int(df3.loc[i][-1])]
df4['Labels']=kmeans.predict(df4)
for i in range(len(df4)):
    df4.loc[i,'Labels']=cluster_modes[int(df4.loc[i][-1])]

In [ ]:
rms_kmeans = sqrt(mean_squared_error(abst, df4['Labels']))
print('RMS error for k=%d is: %d hours'%(optimal_k,rms_kmeans))

In [ ]:
print("For the K Nearest Neighbours classification model:")
print("Accuracy (percentage) = ",knn_acc)
print("Root Mean Squared Error = ",rms_knn)
print("For the K Means clustering model:")
print("Accuracy (percentage) = ",kmeans_acc)
print("Root Mean Squared Error = ",rms_kmeans)

### We see that for this dataset KNN classification performs better than K Means clustering. This could be due to the fact that class labels exist in the data provided for the value of 'Absenteeism time in hours' and hence it might be better to use supervised learning techniques.